# All oscillation analysis

Brief 1-2 sentence description of notebook.

In [1]:
# Imports of all used packages and libraries
import sys
import os
import git
import glob
from collections import defaultdict

In [2]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [3]:
git_root

'/nancy/user/riwata/projects/reward_comp_ext'

In [4]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
import os
import collections
import itertools
from collections import defaultdict
from itertools import combinations

In [7]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import mannwhitneyu


In [8]:
import matplotlib
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import colorsys

In [9]:
from sklearn.metrics import confusion_matrix

In [10]:
from spectral_connectivity import Multitaper, Connectivity
import spectral_connectivity

In [11]:
import utilities.helper
import trodes.read_exported

In [12]:
FONTSIZE = 20

In [13]:
font = {'weight' : 'medium',
        'size'   : 20}

matplotlib.rc('font', **font)

## Inputs & Data

In [14]:
EPHYS_SAMPLE_RATE = 20000

Explanation of each input and where it comes from.

In [15]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc/" # where data is saved should always be shown in the inputs
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = "rce_pilot_3_alone_comp"

In [16]:
# TRIAL_LABELS_DF = pd.read_excel("/blue/npadillacoreano/ryoi360/projects/reward_comp/repos/reward_comp_ext/results/2024_06_26_sleap_clustering/data/rce_pilot_3_alone_comp_per_video_trial_labels.xlsx")
TRIAL_LABELS_DF = pd.read_excel("../../data/{}_per_video_trial_labels.xlsx".format(OUTPUT_PREFIX))

In [17]:
TRIAL_LABELS_DF = TRIAL_LABELS_DF.drop(columns=[col for col in TRIAL_LABELS_DF if "unnamed" in col.lower()], errors="ignore")

In [18]:
TRIAL_LABELS_DF = TRIAL_LABELS_DF.rename(columns={col: col.strip() for col in TRIAL_LABELS_DF}, errors="ignore")

In [19]:
# LFP_SPECTRAL_DF = pd.read_pickle("./proc/{}_03_spectral_bands.pkl".format(OUTPUT_PREFIX))
LFP_SPECTRAL_DF = pd.read_pickle("../2024_11_18_add_sleap/proc/{}_04_lfp_spike_sleap.pkl".format(OUTPUT_PREFIX))
# /nancy/user/riwata/projects/reward_comp_ext/results../../2024_11_18_add_sleap/proc/rce_pilot_3_alone_comp_04_lfp_spike_sleap.pkl"

In [20]:
# LFP_SPECTRAL_DF = pd.read_pickle("./proc/{}_04_spectral_and_sleap.pkl".format(OUTPUT_PREFIX))

In [21]:
LFP_SPECTRAL_DF = LFP_SPECTRAL_DF.drop(columns=[col for col in LFP_SPECTRAL_DF if "unnamed" in col.lower()], errors="ignore")

In [22]:
LFP_SPECTRAL_DF = LFP_SPECTRAL_DF.rename(columns={col: col.strip() for col in LFP_SPECTRAL_DF}, errors="ignore")

In [23]:
# cluster_range_columns = pd.read_pickle("../2024_06_26_sleap_clustering/proc/cluster_range_columns.pkl")

In [24]:
# cluster_range_columns

In [25]:
FULL_LFP_TRACES_PKL = "{}_10_per_trial_lfp_spikes.pkl".format(OUTPUT_PREFIX)

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

## Functions 

In [26]:
def filter_spike_times(arr_2d, start, stop, padding=-1):
    """
    Filters a 2D array based on a start and stop condition and pads shorter arrays to match the length of the longest array.

    Parameters:
    arr_2d (numpy.ndarray): 2D array to be filtered and padded.
    start (int or float): Lower bound of the condition.
    stop (int or float): Upper bound of the condition.
    padding (int or float): Value to use for padding, defaults to -1.

    Returns:
    numpy.ndarray: A 2D array where each inner array has been filtered based on the condition and padded to match the length of the longest array.
    """

    # Filter each row in arr_2d based on the condition
    masked_data = [row[(row >= start) & (row < stop)] for row in arr_2d]

    # Determine the maximum length of arrays in the list
    max_length = max(len(arr) for arr in masked_data)

    # Pad each array in masked_data with the padding value so they all have the same length (max_length)
    padded_arrays = [np.concatenate([x, np.full([max_length - len(x)], padding)]) for x in masked_data]

    # Convert the list of arrays to a 2D numpy array
    padded_arrays = np.array(padded_arrays)

    return padded_arrays

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [27]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

# Ideally functions are defined here first and then data is processed using the functions

# function names are short and in snake case all lowercase
# a function name should be unique but does not have to describe the function
# doc strings describe functions not function names




In [28]:
LFP_SPECTRAL_DF.columns

Index(['cohort', 'session_dir', 'tone_frames', 'box_1_port_entry_frames',
       'box_2_port_entry_frames', 'video_name', 'notes', 'session_path',
       'recording', 'current_subject', 'subject', 'all_subjects',
       'first_timestamp', 'last_timestamp', 'video_timestamps',
       'tone_timestamps', 'box_1_port_entry_timestamps',
       'box_2_port_entry_timestamps', 'lfp_timestamps', 'vHPC_lfp_trace',
       'BLA_lfp_trace', 'LH_lfp_trace', 'MD_lfp_trace', 'mPFC_lfp_trace',
       'recording_name', 'spike_clusters', 'spike_times', 'neuron_average_fr',
       'neuron_average_timestamps', 'start_frame', 'stop_frame',
       'tracked_subject', 'in_video_subjects', 'box_number', 'sleap_name',
       'body_parts', 'box_top_right', 'box_bottom_right', 'box_top_left',
       'reward_port', 'box_bottom_left', 'agent', 'subject_locations',
       'frame_index', 'agent_locations', 'subject_thorax_velocity',
       'agent_thorax_velocity', 'subject_thorax_to_reward_port',
       'agent_thorax_

In [29]:
TRIAL_LABELS_DF.columns

Index(['session_dir', 'tone_start_frame', 'reward_start', 'reward_dispensed',
       'tone_stop_frame', 'condition', 'competition_closeness',
       'get_reward_frame', 'out_reward_frame', 'notes', 'experiment',
       'box_1_port_entry_frames', 'box_2_port_entry_frames', 'video_name',
       'all_subjects', 'first_timestamp', 'last_timestamp',
       'tone_start_timestamp', 'tone_stop_timestamp',
       'box_1_port_entry_timestamps', 'box_2_port_entry_timestamps'],
      dtype='object')

## Merging the labels and LFP spectral

In [30]:
TRIAL_LABELS_DF.head()

,session_dir,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,notes,...,box_1_port_entry_frames,box_2_port_entry_frames,video_name,all_subjects,first_timestamp,last_timestamp,tone_start_timestamp,tone_stop_timestamp,box_1_port_entry_timestamps,box_2_port_entry_timestamps
0,20240320_142408_alone_comp_subj_3-1_and_3-3,0.0,80,100.0,74.0,NaN,NaN,NaN,NaN,NaN,...,[ 0 74],[ 0 74],20240320_142408_alone_comp_subj_3-1_and_3-3.1....,"['3.1', '3.3']",1830734.0,65425515.0,0.0,75474.0,[ 0 75474],[ 0 75474]
1,20240320_142408_alone_comp_subj_3-1_and_3-3,1272.0,1352,1372.0,1471.0,rewarded,NaN,NaN,NaN,NaN,...,[1282 1283] [1285 1297] [1326 1344] [1362 1509],[1326 1328] [1331 1347] [1380 1592],20240320_142408_alone_comp_subj_3-1_and_3-3.1....,"['3.1', '3.3']",1830734.0,65425515.0,1275889.0,1475891.0,[1285888 1286688] [1287488 1300689] [1330286 1...,[1329489 1331487] [1334489 1350089] [1383490 1...
2,20240320_142408_alone_comp_subj_3-1_and_3-3,3268.0,3348,3368.0,3469.0,rewarded,NaN,NaN,NaN,NaN,...,[3281 3324] [3324 3327] [3344 3347] [3347 3523],[3327 3704],20240320_142408_alone_comp_subj_3-1_and_3-3.1....,"['3.1', '3.3']",1830734.0,65425515.0,3275911.0,3475914.0,[3287914 3330314] [3331114 3333712] [3351715 3...,[3333515 3712519]
3,20240320_142408_alone_comp_subj_3-1_and_3-3,4466.0,4546,4566.0,4667.0,rewarded,NaN,NaN,NaN,NaN,...,[4441 4476] [4476 4489] [4508 4509] [4511 4534...,[4445 4473] [4494 4551] [4566 4830],20240320_142408_alone_comp_subj_3-1_and_3-3.1....,"['3.1', '3.3']",1830734.0,65425515.0,4475927.0,4675931.0,[4450528 4485129] [4485929 4497529] [4516929 4...,[4454926 4483129] [4503531 4560730] [4575330 4...
4,20240320_142408_alone_comp_subj_3-1_and_3-3,5465.0,5545,5565.0,5664.0,rewarded,NaN,NaN,NaN,NaN,...,[5483 5512] [5552 5698],[5455 5486] [5581 5794],20240320_142408_alone_comp_subj_3-1_and_3-3.1....,"['3.1', '3.3']",1830734.0,65425515.0,5475942.0,5675942.0,[5493742 5523542] [5563742 5709147],[5466941 5496741] [5592343 5806546]


In [31]:
LFP_SPECTRAL_DF.head()

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,notes,session_path,recording,current_subject,...,reward_port,box_bottom_left,agent,subject_locations,frame_index,agent_locations,subject_thorax_velocity,agent_thorax_velocity,subject_thorax_to_reward_port,agent_thorax_to_reward_port
0,rce_pilot_3,20240320_142408_alone_comp_subj_3-1_and_3-3,"[[0, 74], [1272, 1471], [3268, 3469], [4466, 4...","[[0, 74], [152, 157], [252, 305], [305, 323], ...","[[0, 74], [160, 225], [254, 257], [257, 279], ...",20240320_142408_alone_comp_subj_3-1_and_3-3.1,NaN,/scratch/back_up/reward_competition_extention/...,20240320_142408_alone_comp_subj_3-1_t6b6_merged,3.1,...,"[60.89621072293514, 35.03865285708705]","[45.88328858282252, 33.462113987746186]",3.3,"[[[80.72344548957854, 15.18291385091661], [82....","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[[78.76350635960564, 15.794081734700777], [82...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[11.09, 10.26, 9.484, 8.77, 8.125, 7.555, 7.05...","[26.84, 26.84, 26.84, 26.84, 26.84, 26.84, 26....","[22.83, 23.27, 23.69, 24.17, 24.75, 24.6, 24.3..."
1,rce_pilot_3,20240320_142408_alone_comp_subj_3-1_and_3-3,"[[0, 74], [1272, 1471], [3268, 3469], [4466, 4...","[[0, 74], [152, 157], [252, 305], [305, 323], ...","[[0, 74], [160, 225], [254, 257], [257, 279], ...",20240320_142408_alone_comp_subj_3-1_and_3-3.1,NaN,/scratch/back_up/reward_competition_extention/...,20240320_142408_alone_comp_subj_3-3_t5b5_merged,3.3,...,"[60.89621072293514, 35.03865285708705]","[45.88328858282252, 33.462113987746186]",3.1,"[[[78.76350635960564, 15.794081734700777], [82...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[[80.72344548957854, 15.18291385091661], [82....","[11.09, 10.26, 9.484, 8.77, 8.125, 7.555, 7.05...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[22.83, 23.27, 23.69, 24.17, 24.75, 24.6, 24.3...","[26.84, 26.84, 26.84, 26.84, 26.84, 26.84, 26...."
2,rce_pilot_3,20240320_171038_alone_comp_subj_4-2_and_4-3,"[[0, 79], [1276, 1477], [3272, 3473], [4470, 4...","[[0, 79], [84, 92], [92, 132], [145, 175], [25...","[[0, 79], [79, 186], [188, 191], [191, 268], [...",20240320_171038_alone_comp_subj_4-2_and_4-3.1,NaN,/scratch/back_up/reward_competition_extention/...,20240320_171038_alone_comp_subj_4-2_t6b6_merged,4.2,...,"[61.213420932957966, 35.084751405625006]","[46.2961822248657, 33.41760470505085]",4.3,"[[[64.90703525371228, 33.28528990033932], [47....","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[[64.44770418343036, 33.62145921311694], [60....","[4.75, 4.48, 4.207, 3.932, 3.658, 3.383, 3.105...","[2.611, 2.268, 1.943, 1.642, 1.36, 1.101, 0.86...","[4.5, 4.5, 4.5, 4.5, 4.5, 4.5, 4.5, 4.5, 4.5, ...","[4.754, 4.723, 4.504, 4.598, 4.617, 4.617, 4.4..."
3,rce_pilot_3,20240320_171038_alone_comp_subj_4-2_and_4-3,"[[0, 79], [1276, 1477], [3272, 3473], [4470, 4...","[[0, 79], [84, 92], [92, 132], [145, 175], [25...","[[0, 79], [79, 186], [188, 191], [191, 268], [...",20240320_171038_alone_comp_subj_4-2_and_4-3.1,NaN,/scratch/back_up/reward_competition_extention/...,20240320_171038_alone_comp_subj_4-3_t5b5_merged,4.3,...,"[61.213420932957966, 35.084751405625006]","[46.2961822248657, 33.41760470505085]",4.2,"[[[64.44770418343036, 33.62145921311694], [60....","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[[64.90703525371228, 33.28528990033932], [47....","[2.611, 2.268, 1.943, 1.642, 1.36, 1.101, 0.86...","[4.75, 4.48, 4.207, 3.932, 3.658, 3.383, 3.105...","[4.754, 4.723, 4.504, 4.598, 4.617, 4.617, 4.4...","[4.5, 4.5, 4.5, 4.5, 4.5, 4.5, 4.5, 4.5, 4.5, ..."
4,rce_pilot_3,20240322_120625_alone_comp_subj_3-3_and_3-4,"[[0, 101], [1299, 1500], [3297, 3496], [4494, ...","[[0, 101], [102, 105], [181, 226], [226, 260],...","[[0, 101], [223, 248], [286, 292], [293, 302],...",20240322_120625_alone_comp_subj_3-3_and_3-4.1,NaN,/scratch/back_up/reward_competition_extention/...,20240322_120625_alone_comp_subj_3-3_t6b6_merged,3.3,...,"[60.64564060702992, 3

# Dropping redundant columns

In [32]:
dropping_columns = list(set(TRIAL_LABELS_DF.columns).intersection(set(LFP_SPECTRAL_DF.columns)))

In [33]:
dropping_columns

['box_2_port_entry_timestamps',
 'all_subjects',
 'first_timestamp',
 'video_name',
 'session_dir',
 'last_timestamp',
 'box_1_port_entry_timestamps',
 'notes',
 'box_2_port_entry_frames',
 'box_1_port_entry_frames']

In [34]:
TRIAL_LABELS_DF = TRIAL_LABELS_DF.drop(columns=[col for col in dropping_columns if col != "video_name"], errors="ignore")

## Merging the LFP and trial dataframe

In [35]:
LFP_SPECTRAL_DF["video_name"] = LFP_SPECTRAL_DF["video_name"].apply(lambda x: x.strip(".videoTimeStamps.cameraHWSync"))

In [36]:
LFP_SPECTRAL_DF["video_name"].unique()

array(['20240320_142408_alone_comp_subj_3-1_and_3-3.1',
       '20240320_171038_alone_comp_subj_4-2_and_4-3.1',
       '20240322_120625_alone_comp_subj_3-3_and_3-4.1',
       '20240322_160946_alone_comp_subj_4-3_and_4-4.1',
       '20240323_122227_alone_comp_subj_5-2_and_5-3.1',
       '20240323_144517_alone_comp_subj_3-1_and_3-4.1',
       '20240323_165815_alone_comp_subj_4-2_and_4-4.1'], dtype=object)

In [37]:
LFP_SPECTRAL_DF["current_subject"] = LFP_SPECTRAL_DF["current_subject"].astype(str)


In [38]:
LFP_SPECTRAL_DF["current_subject"].head()

0    3.1
1    3.3
2    4.2
3    4.3
4    3.3
Name: current_subject, dtype: object

In [39]:
TRIAL_LABELS_DF = TRIAL_LABELS_DF.dropna(subset=["video_name"])

In [40]:
TRIAL_LABELS_DF = TRIAL_LABELS_DF.dropna(subset=["condition"])

In [41]:
TRIAL_LABELS_DF["video_name"] = TRIAL_LABELS_DF["video_name"].apply(lambda x: x.strip(".videoTimeStamps.cameraHWSync"))


In [42]:
TRIAL_LABELS_DF["video_name"].unique()

array(['20240320_142408_alone_comp_subj_3-1_and_3-3.1',
       '20240320_142408_alone_comp_subj_3-1_and_3-3.2',
       '20240320_171038_alone_comp_subj_4-2_and_4-3.1',
       '20240320_171038_alone_comp_subj_4-2_and_4-3.2',
       '20240322_120625_alone_comp_subj_3-3_and_3-4.1',
       '20240322_120625_alone_comp_subj_3-3_and_3-4.2',
       '20240322_160946_alone_comp_subj_4-3_and_4-4.1',
       '20240322_160946_alone_comp_subj_4-3_and_4-4.2',
       '20240323_122227_alone_comp_subj_5-2_and_5-3.1',
       '20240323_122227_alone_comp_subj_5-2_and_5-3.2',
       '20240323_144517_alone_comp_subj_3-1_and_3-4.1',
       '20240323_144517_alone_comp_subj_3-1_and_3-4.2',
       '20240323_165815_alone_comp_subj_4-2_and_4-4.1',
       '20240323_165815_alone_comp_subj_4-2_and_4-4.2'], dtype=object)

In [43]:
TRIALS_AND_SPECTRAL_DF = pd.merge(TRIAL_LABELS_DF, LFP_SPECTRAL_DF, on=["video_name"], how="inner", suffixes=('', '_remove'))

In [44]:
TRIALS_AND_SPECTRAL_DF[[col for col in TRIALS_AND_SPECTRAL_DF if "_remove" in col]].head()

""
0
1
2
3
4


In [45]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=[col for col in TRIALS_AND_SPECTRAL_DF if "remove" in col], errors="ignore")

In [46]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=[col for col in TRIALS_AND_SPECTRAL_DF if "Unnamed" in col])

In [47]:
TRIALS_AND_SPECTRAL_DF.head()

,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,experiment,video_name,...,reward_port,box_bottom_left,agent,subject_locations,frame_index,agent_locations,subject_thorax_velocity,agent_thorax_velocity,subject_thorax_to_reward_port,agent_thorax_to_reward_port
0,1272.0,1352,1372.0,1471.0,rewarded,NaN,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,"[60.89621072293514, 35.03865285708705]","[45.88328858282252, 33.462113987746186]",3.3,"[[[80.72344548957854, 15.18291385091661], [82....","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[[78.76350635960564, 15.794081734700777], [82...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[11.09, 10.26, 9.484, 8.77, 8.125, 7.555, 7.05...","[26.84, 26.84, 26.84, 26.84, 26.84, 26.84, 26....","[22.83, 23.27, 23.69, 24.17, 24.75, 24.6, 24.3..."
1,1272.0,1352,1372.0,1471.0,rewarded,NaN,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,"[60.89621072293514, 35.03865285708705]","[45.88328858282252, 33.462113987746186]",3.1,"[[[78.76350635960564, 15.794081734700777], [82...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[[80.72344548957854, 15.18291385091661], [82....","[11.09, 10.26, 9.484, 8.77, 8.125, 7.555, 7.05...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[22.83, 23.27, 23.69, 24.17, 24.75, 24.6, 24.3...","[26.84, 26.84, 26.84, 26.84, 26.84, 26.84, 26...."
2,3268.0,3348,3368.0,3469.0,rewarded,NaN,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,"[60.89621072293514, 35.03865285708705]","[45.88328858282252, 33.462113987746186]",3.3,"[[[80.72344548957854, 15.18291385091661], [82....","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[[78.76350635960564, 15.794081734700777], [82...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[11.09, 10.26, 9.484, 8.77, 8.125, 7.555, 7.05...","[26.84, 26.84, 26.84, 26.84, 26.84, 26.84, 26....","[22.83, 23.27, 23.69, 24.17, 24.75, 24.6, 24.3..."
3,3268.0,3348,3368.0,3469.0,rewarded,NaN,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,"[60.89621072293514, 35.03865285708705]","[45.88328858282252, 33.462113987746186]",3.1,"[[[78.76350635960564, 15.794081734700777], [82...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[[80.72344548957854, 15.18291385091661], [82....","[11.09, 10.26, 9.484, 8.77, 8.125, 7.555, 7.05...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[22.83, 23.27, 23.69, 24.17, 24.75, 24.6, 24.3...","[26.84, 26.84, 26.84, 26.84, 26.84, 26.84, 26...."
4,4466.0,4546,4566.0,4667.0,rewarded,NaN,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,"[60.89621072293514, 35.03865285708705]","[45.88328858282252, 33.462113987746186]",3.3,"[[[80.72344548957854, 15.18291385091661], [82....","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[[78.76350635960564, 15.794081734700777], [82...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[11.09, 10.26, 9.484, 8.77, 8.125, 7.555, 7.05...","[26.84, 26.84, 26.84, 26.84, 26.84, 26.84, 26....","[22.83, 23.27, 23.69, 24.17, 24.75, 24.6, 24.3..."


In [48]:
TRIALS_AND_SPECTRAL_DF.tail()

,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,experiment,video_name,...,reward_port,box_bottom_left,agent,subject_locations,frame_index,agent_locations,subject_thorax_velocity,agent_thorax_velocity,subject_thorax_to_reward_port,agent_thorax_to_reward_port
454,55484.0,55564,55584.0,55685.0,4.4,Subj 2 then Subj 1,NaN,NaN,alone_comp,20240323_165815_alone_comp_subj_4-2_and_4-4.1,...,"[60.894370390068914, 34.253301735040985]","[45.79835911412773, 32.60866761368506]",4.4,"[[[56.986964177053565, 31.8836904164058], [55....","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[[61.45843466863499, 32.032592967139145], [59...","[13.48, 12.05, 10.69, 9.39, 8.164, 6.996, 5.9,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[9.25, 9.18, 8.94, 8.69, 8.21, 7.492, 6.39, 5....","[27.78, 27.78, 27.78, 27.78, 27.78, 27.78, 27...."
455,56483.0,56563,56583.0,56683.0,4.4,Subj 2 then Subj 1,NaN,NaN,alone_comp,20240323_165815_alone_comp_subj_4-2_and_4-4.1,...,"[60.894370390068914, 34.253301735040985]","[45.79835911412773, 32.60866761368506]",4.4,"[[[56.986964177053565, 31.8836904164058], [55....","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[[61.45843466863499, 32.032592967139145], [59...","[13.48, 12.05, 10.69, 9.39, 8.164, 6.996, 5.9,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[9.25, 9.18, 8.94, 8.69, 8.21, 7.492, 6.39, 5....","[27.78, 27.78, 27.78, 27.78, 27.78, 27.78, 27...."
456,57880.0,57960,57980.0,58079.0,4.4,Subj 2 then Subj 1,NaN,NaN,alone_comp,20240323_165815_alone_comp_subj_4-2_and_4-4.1,...,"[60.894370390068914, 34.253301735040985]","[45.79835911412773, 32.60866761368506]",4.4,"[[[56.986964177053565, 31.8836904164058], [55....","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[[61.45843466863499, 32.032592967139145], [59...","[13.48, 12.05, 10.69, 9.39, 8.164, 6.996, 5.9,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[9.25, 9.18, 8.94, 8.69, 8.21, 7.492, 6.39, 5....","[27.78, 27.78, 27.78, 27.78, 27.78, 27.78, 27...."
457,60077.0,60157,60177.0,60276.0,4.4,Subj 2 blocking Subj 1,NaN,NaN,alone_comp,20240323_165815_alone_comp_subj_4-2_and_4-4.1,...,"[60.894370390068914, 34.253301735040985]","[45.79835911412773, 32.60866761368506]",4.4,"[[[56.986964177053565, 31.8836904164058], [55....","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[[61.45843466863499, 32.032592967139145], [59...","[13.48, 12.05, 10.69, 9.39, 8.164, 6.996, 5.9,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[9.25, 9.18, 8.94, 8.69, 8.21, 7.492, 6.39, 5....","[27.78, 27.78, 27.78, 27.78, 27.78, 27.78, 27...."
458,61174.0,61254,61274.0,61375.0,tie,Close Call,NaN,NaN,alone_comp,20240323_165815_alone_comp_subj_4-2_and_4-4.1,...,"[60.894370390068914, 34.253301735040985]","[45.79835911412773, 32.60866761368506]",4.4,"[[[56.986964177053565, 31.8836904164058], [55....","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[[61.45843466863499, 32.032592967139145], [59...","[13.48, 12.05, 10.69, 9.39, 8.164, 6.996, 5.9,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[9.25, 9.18, 8.94, 8.69, 8.21, 7.492, 6.39, 5....","[27.78, 27.78, 27.78, 27.78, 27.78, 27.78, 27...."


In [49]:
timestamps_col = [col for col in TRIALS_AND_SPECTRAL_DF if "timestamp" in col and "timestamps" not in col and "cluster" not in col]

In [50]:
for col in timestamps_col:
    TRIALS_AND_SPECTRAL_DF[col] = TRIALS_AND_SPECTRAL_DF[col].astype(int)

In [51]:
TRIALS_AND_SPECTRAL_DF[timestamps_col].head()

,tone_start_timestamp,tone_stop_timestamp,first_timestamp,last_timestamp
0,1275889,1475891,1830734,65425515
1,1275889,1475891,1830734,65425515
2,3275911,3475914,1830734,65425515
3,3275911,3475914,1830734,65425515
4,4475927,4675931,1830734,65425515


In [52]:
frame_col = [col for col in TRIALS_AND_SPECTRAL_DF if "frame" in col and "tone" in col and "frames" not in col]

In [53]:
for col in frame_col:
    TRIALS_AND_SPECTRAL_DF[col] = TRIALS_AND_SPECTRAL_DF[col].astype(int)

In [54]:
TRIALS_AND_SPECTRAL_DF[frame_col].head()

,tone_start_frame,tone_stop_frame
0,1272,1471
1,1272,1471
2,3268,3469
3,3268,3469
4,4466,4667


## Adding the SLEAP data

In [55]:
TRIALS_AND_SPECTRAL_DF.head()

,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,experiment,video_name,...,reward_port,box_bottom_left,agent,subject_locations,frame_index,agent_locations,subject_thorax_velocity,agent_thorax_velocity,subject_thorax_to_reward_port,agent_thorax_to_reward_port
0,1272,1352,1372.0,1471,rewarded,NaN,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,"[60.89621072293514, 35.03865285708705]","[45.88328858282252, 33.462113987746186]",3.3,"[[[80.72344548957854, 15.18291385091661], [82....","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[[78.76350635960564, 15.794081734700777], [82...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[11.09, 10.26, 9.484, 8.77, 8.125, 7.555, 7.05...","[26.84, 26.84, 26.84, 26.84, 26.84, 26.84, 26....","[22.83, 23.27, 23.69, 24.17, 24.75, 24.6, 24.3..."
1,1272,1352,1372.0,1471,rewarded,NaN,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,"[60.89621072293514, 35.03865285708705]","[45.88328858282252, 33.462113987746186]",3.1,"[[[78.76350635960564, 15.794081734700777], [82...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[[80.72344548957854, 15.18291385091661], [82....","[11.09, 10.26, 9.484, 8.77, 8.125, 7.555, 7.05...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[22.83, 23.27, 23.69, 24.17, 24.75, 24.6, 24.3...","[26.84, 26.84, 26.84, 26.84, 26.84, 26.84, 26...."
2,3268,3348,3368.0,3469,rewarded,NaN,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,"[60.89621072293514, 35.03865285708705]","[45.88328858282252, 33.462113987746186]",3.3,"[[[80.72344548957854, 15.18291385091661], [82....","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[[78.76350635960564, 15.794081734700777], [82...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[11.09, 10.26, 9.484, 8.77, 8.125, 7.555, 7.05...","[26.84, 26.84, 26.84, 26.84, 26.84, 26.84, 26....","[22.83, 23.27, 23.69, 24.17, 24.75, 24.6, 24.3..."
3,3268,3348,3368.0,3469,rewarded,NaN,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,"[60.89621072293514, 35.03865285708705]","[45.88328858282252, 33.462113987746186]",3.1,"[[[78.76350635960564, 15.794081734700777], [82...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[[80.72344548957854, 15.18291385091661], [82....","[11.09, 10.26, 9.484, 8.77, 8.125, 7.555, 7.05...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[22.83, 23.27, 23.69, 24.17, 24.75, 24.6, 24.3...","[26.84, 26.84, 26.84, 26.84, 26.84, 26.84, 26...."
4,4466,4546,4566.0,4667,rewarded,NaN,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,"[60.89621072293514, 35.03865285708705]","[45.88328858282252, 33.462113987746186]",3.3,"[[[80.72344548957854, 15.18291385091661], [82....","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[[78.76350635960564, 15.794081734700777], [82...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[11.09, 10.26, 9.484, 8.77, 8.125, 7.555, 7.05...","[26.84, 26.84, 26.84, 26.84, 26.84, 26.84, 26....","[22.83, 23.27, 23.69, 24.17, 24.75, 24.6, 24.3..."


In [56]:
TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"].head()

0    1275889
1    1275889
2    3275911
3    3275911
4    4475927
Name: tone_start_timestamp, dtype: int64

In [57]:
TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"] = TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"].astype(int)

In [58]:
TRIALS_AND_SPECTRAL_DF["current_subject"] = TRIALS_AND_SPECTRAL_DF["current_subject"].apply(lambda x: str(x).strip())

# Calculating the relevant timestamps

In [59]:
TRIALS_AND_SPECTRAL_DF["baseline_start_timestamp"] = TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"] - 10 * EPHYS_SAMPLE_RATE

In [60]:
TRIALS_AND_SPECTRAL_DF["post_trial_end_timestamp"] = TRIALS_AND_SPECTRAL_DF["tone_stop_timestamp"] + 10 * EPHYS_SAMPLE_RATE

In [61]:
TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"].head()

0    1275889
1    1275889
2    3275911
3    3275911
4    4475927
Name: tone_start_timestamp, dtype: int64

In [62]:
TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"].head()

0    1275889
1    1275889
2    3275911
3    3275911
4    4475927
Name: tone_start_timestamp, dtype: int64

In [63]:
TRIALS_AND_SPECTRAL_DF["tone_stop_timestamp"].head()

0    1475891
1    1475891
2    3475914
3    3475914
4    4675931
Name: tone_stop_timestamp, dtype: int64

In [64]:
TRIALS_AND_SPECTRAL_DF.head()

,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,experiment,video_name,...,agent,subject_locations,frame_index,agent_locations,subject_thorax_velocity,agent_thorax_velocity,subject_thorax_to_reward_port,agent_thorax_to_reward_port,baseline_start_timestamp,post_trial_end_timestamp
0,1272,1352,1372.0,1471,rewarded,NaN,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,3.3,"[[[80.72344548957854, 15.18291385091661], [82....","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[[78.76350635960564, 15.794081734700777], [82...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[11.09, 10.26, 9.484, 8.77, 8.125, 7.555, 7.05...","[26.84, 26.84, 26.84, 26.84, 26.84, 26.84, 26....","[22.83, 23.27, 23.69, 24.17, 24.75, 24.6, 24.3...",1075889,1675891
1,1272,1352,1372.0,1471,rewarded,NaN,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,3.1,"[[[78.76350635960564, 15.794081734700777], [82...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[[80.72344548957854, 15.18291385091661], [82....","[11.09, 10.26, 9.484, 8.77, 8.125, 7.555, 7.05...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[22.83, 23.27, 23.69, 24.17, 24.75, 24.6, 24.3...","[26.84, 26.84, 26.84, 26.84, 26.84, 26.84, 26....",1075889,1675891
2,3268,3348,3368.0,3469,rewarded,NaN,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,3.3,"[[[80.72344548957854, 15.18291385091661], [82....","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[[78.76350635960564, 15.794081734700777], [82...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[11.09, 10.26, 9.484, 8.77, 8.125, 7.555, 7.05...","[26.84, 26.84, 26.84, 26.84, 26.84, 26.84, 26....","[22.83, 23.27, 23.69, 24.17, 24.75, 24.6, 24.3...",3075911,3675914
3,3268,3348,3368.0,3469,rewarded,NaN,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,3.1,"[[[78.76350635960564, 15.794081734700777], [82...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[[80.72344548957854, 15.18291385091661], [82....","[11.09, 10.26, 9.484, 8.77, 8.125, 7.555, 7.05...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[22.83, 23.27, 23.69, 24.17, 24.75, 24.6, 24.3...","[26.84, 26.84, 26.84, 26.84, 26.84, 26.84, 26....",3075911,3675914
4,4466,4546,4566.0,4667,rewarded,NaN,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,3.3,"[[[80.72344548957854, 15.18291385091661], [82....","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[[78.76350635960564, 15.794081734700777], [82...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[11.09, 10.26, 9.484, 8.77, 8.125, 7.555, 7.05...","[26.84, 26.84, 26.84, 26.84, 26.84, 26.84, 26....","[22.83, 23.27, 23.69, 24.17, 24.75, 24.6, 24.3...",4275927,4875931


# Filtering out the SLEAP posed for during trials

In [65]:
TRIALS_AND_SPECTRAL_DF["video_frame"] = TRIALS_AND_SPECTRAL_DF["video_timestamps"].apply(lambda x: np.array(list(range(len(x)))) + 1)

In [66]:
TRIALS_AND_SPECTRAL_DF["video_frame"].head().apply(lambda x: x.shape)

0    (63504,)
1    (63504,)
2    (63504,)
3    (63504,)
4    (63504,)
Name: video_frame, dtype: object

In [67]:
sleap_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if ("thorax" in col or "nose" in col or "standard_embedding" in col or "frame_index" in col or "locations" in col or "tailbase" in col or "competitiveness" in col) and "timestamp" not in col and "dict" not in col]


In [68]:
sleap_columns

['subject_locations',
 'frame_index',
 'agent_locations',
 'subject_thorax_velocity',
 'agent_thorax_velocity',
 'subject_thorax_to_reward_port',
 'agent_thorax_to_reward_port']

In [69]:
for col in sleap_columns:
    print(col)
    print(TRIALS_AND_SPECTRAL_DF[TRIALS_AND_SPECTRAL_DF["condition"] != "rewarded"][col].apply(lambda x: x.shape if isinstance(x, np.ndarray) else np.nan).head())

subject_locations
42    (63504, 6, 2)
43    (63504, 6, 2)
44    (63504, 6, 2)
45    (63504, 6, 2)
46    (63504, 6, 2)
Name: subject_locations, dtype: object
frame_index
42    (63504,)
43    (63504,)
44    (63504,)
45    (63504,)
46    (63504,)
Name: frame_index, dtype: object
agent_locations
42    (63504, 6, 2)
43    (63504, 6, 2)
44    (63504, 6, 2)
45    (63504, 6, 2)
46    (63504, 6, 2)
Name: agent_locations, dtype: object
subject_thorax_velocity
42    (63504,)
43    (63504,)
44    (63504,)
45    (63504,)
46    (63504,)
Name: subject_thorax_velocity, dtype: object
agent_thorax_velocity
42    (63504,)
43    (63504,)
44    (63504,)
45    (63504,)
46    (63504,)
Name: agent_thorax_velocity, dtype: object
subject_thorax_to_reward_port
42    (63504,)
43    (63504,)
44    (63504,)
45    (63504,)
46    (63504,)
Name: subject_thorax_to_reward_port, dtype: object
agent_thorax_to_reward_port
42    (63504,)
43    (63504,)
44    (63504,)
45    (63504,)
46    (63504,)
Name: agent_thorax_to_rewar

In [70]:
TRIALS_AND_SPECTRAL_DF.columns

Index(['tone_start_frame', 'reward_start', 'reward_dispensed',
       'tone_stop_frame', 'condition', 'competition_closeness',
       'get_reward_frame', 'out_reward_frame', 'experiment', 'video_name',
       'tone_start_timestamp', 'tone_stop_timestamp', 'cohort', 'session_dir',
       'tone_frames', 'box_1_port_entry_frames', 'box_2_port_entry_frames',
       'notes', 'session_path', 'recording', 'current_subject', 'subject',
       'all_subjects', 'first_timestamp', 'last_timestamp', 'video_timestamps',
       'tone_timestamps', 'box_1_port_entry_timestamps',
       'box_2_port_entry_timestamps', 'lfp_timestamps', 'vHPC_lfp_trace',
       'BLA_lfp_trace', 'LH_lfp_trace', 'MD_lfp_trace', 'mPFC_lfp_trace',
       'recording_name', 'spike_clusters', 'spike_times', 'neuron_average_fr',
       'neuron_average_timestamps', 'start_frame', 'stop_frame',
       'tracked_subject', 'in_video_subjects', 'box_number', 'sleap_name',
       'body_parts', 'box_top_right', 'box_bottom_right', 'box_t

In [71]:
for col in sorted(sleap_columns):
    updated_item_col = "trial_{}".format(col)
    print(updated_item_col)
    updated_timestamp_col = "trial_video_timestamps"
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["video_timestamps"], items=x[col])[1] if isinstance(x["video_timestamps"], np.ndarray) else np.nan, axis=1)

TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["video_timestamps"], items=x[col])[0] if isinstance(x["video_timestamps"], np.ndarray) else np.nan, axis=1)

trial_agent_locations
trial_agent_thorax_to_reward_port
trial_agent_thorax_velocity
trial_frame_index
trial_subject_locations
trial_subject_thorax_to_reward_port
trial_subject_thorax_velocity


In [72]:
for col in sorted(sleap_columns):
    updated_item_col = "baseline_{}".format(col)
    print(updated_item_col)
    updated_timestamp_col = "baseline_video_timestamps"
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["tone_start_timestamp"], timestamps=x["video_timestamps"], items=x[col])[1] if isinstance(x["video_timestamps"], np.ndarray) else np.nan, axis=1)

TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["tone_start_timestamp"], timestamps=x["video_timestamps"], items=x[col])[0] if isinstance(x["video_timestamps"], np.ndarray) else np.nan, axis=1)

baseline_agent_locations
baseline_agent_thorax_to_reward_port
baseline_agent_thorax_velocity
baseline_frame_index
baseline_subject_locations
baseline_subject_thorax_to_reward_port
baseline_subject_thorax_velocity


In [73]:
TRIALS_AND_SPECTRAL_DF[TRIALS_AND_SPECTRAL_DF["condition"] != "rewarded"].head()

,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,experiment,video_name,...,trial_subject_thorax_velocity,trial_video_timestamps,baseline_agent_locations,baseline_agent_thorax_to_reward_port,baseline_agent_thorax_velocity,baseline_frame_index,baseline_subject_locations,baseline_subject_thorax_to_reward_port,baseline_subject_thorax_velocity,baseline_video_timestamps
42,34240,34320,34340.0,34440,3.1,Subj 1 Only,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,"[2.986, 3.566, 4.12, 4.67, 5.203, 5.707, 6.19,...","[34276710, 34278096, 34279482, 34279482, 34280...","[[[54.047201234350496, 32.48462136905939], [52...","[12.51, 12.984, 13.26, 13.516, 12.99, 12.27, 1...","[15.01, 16.55, 18.02, 19.38, 20.64, 21.8, 22.8...","[34041, 34042, 34043, 34044, 34045, 34046, 340...","[[[62.81573570925102, 33.49031610207749], [59....","[4.99, 4.99, 4.992, 4.992, 5.125, 5.0, 4.996, ...","[0.3733, 0.4482, 0.573, 0.782, 1.051, 1.322, 1...","[34077145, 34077145, 34078531, 34079917, 34079..."
43,34240,34320,34340.0,34440,3.1,Subj 1 Only,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,"[16.34, 15.88, 15.375, 14.82, 14.21, 13.56, 12...","[34276710, 34278096, 34279482, 34279482, 34280...","[[[62.81573570925102, 33.49031610207749], [59....","[4.99, 4.99, 4.992, 4.992, 5.125, 5.0, 4.996, ...","[0.3733, 0.4482, 0.573, 0.782, 1.051, 1.322, 1...","[34041, 34042, 34043, 34044, 34045, 34046, 340...","[[[54.047201234350496, 32.48462136905939], [52...","[12.51, 12.984, 13.26, 13.516, 12.99, 12.27, 1...","[15.01, 16.55, 18.02, 19.38, 20.64, 21.8, 22.8...","[34077145, 34077145, 34078531, 34079917, 34079..."
44,35238,35318,35338.0,35437,3.1,Subj 1 then Subj 2,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,"[0.128, 0.07196, 0.05035, 0.1224, 0.2048, 0.27...","[35277308, 35277308, 35278694, 35280080, 35280...","[[[62.32604037851049, 33.47953327548524], [60....","[5.645, 5.66, 5.656, 5.66, 5.84, 5.92, 5.844, ...","[1.193, 1.498, 1.745, 1.959, 2.1, 2.193, 2.254...","[35039, 35040, 35041, 35042, 35043, 35044, 350...","[[[47.349319046024625, 35.41309285214402], [46...","[9.375, 10.15, 11.125, 11.625, 12.086, 12.82, ...","[6.508, 6.81, 6.98, 6.93, 6.676, 6.29, 5.844, ...","[35076357, 35077743, 35079129, 35079129, 35080..."
45,35238,35318,35338.0,35437,3.1,Subj 1 then Subj 2,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,"[18.23, 18.75, 19.03, 19.14, 19.11, 18.97, 18....","[35277308, 35277308, 35278694, 35280080, 35280...","[[[47.349319046024625, 35.41309285214402], [46...","[9.375, 10.15, 11.125, 11.625, 12.086, 12.82, ...","[6.508, 6.81, 6.98, 6.93, 6.676, 6.29, 5.844, ...","[35039, 35040, 35041, 35042, 35043, 35044, 350...","[[[62.32604037851049, 33.47953327548524], [60....","[5.645, 5.66, 5.656, 5.66, 5.84, 5.92, 5.844, ...","[1.193, 1.498, 1.745, 1.959, 2.1, 2.193, 2.254...","[35076357, 35077743, 35079129, 35079129, 35080..."
46,36337,36417,36437.0,36536,3.3,Subj 2 Only,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,"[1.152, 1.122, 1.069, 1.02, 0.977, 0.9233, 0.8...","[36377688, 36379074, 36380460, 36380460, 36381...","[[[65.25330160033462, 8.985615969942064], [61....","[28.55, 28.38, 28.39, 28.38, 28.38, 28.69, 28....","[0.4546, 0.4463, 0.474, 0.5103, 0.5757, 0.665,...","[36137, 36138, 36139, 36140, 36141, 36142, 361...","[[[61.80969837607312, 33.48027678986218], [58....","[4.496, 4.51, 4.52, 4.57, 4.562, 4.55, 4.652, ...","[0.638, 0.669, 0.703, 0.739, 0.7725, 0.8027, 0...","[36176737, 36177403, 36178123, 36179509, 36180..."


In [74]:
TRIALS_AND_SPECTRAL_DF[TRIALS_AND_SPECTRAL_DF["condition"] != "rewarded"]["trial_frame_index"].iloc[0].shape

(200,)

In [75]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=sleap_columns + ["video_timestamps"], errors="ignore")

In [76]:
for col in TRIALS_AND_SPECTRAL_DF.columns:
    print(col)

tone_start_frame
reward_start
reward_dispensed
tone_stop_frame
condition
competition_closeness
get_reward_frame
out_reward_frame
experiment
video_name
tone_start_timestamp
tone_stop_timestamp
cohort
session_dir
tone_frames
box_1_port_entry_frames
box_2_port_entry_frames
notes
session_path
recording
current_subject
subject
all_subjects
first_timestamp
last_timestamp
tone_timestamps
box_1_port_entry_timestamps
box_2_port_entry_timestamps
lfp_timestamps
vHPC_lfp_trace
BLA_lfp_trace
LH_lfp_trace
MD_lfp_trace
mPFC_lfp_trace
recording_name
spike_clusters
spike_times
neuron_average_fr
neuron_average_timestamps
start_frame
stop_frame
tracked_subject
in_video_subjects
box_number
sleap_name
body_parts
box_top_right
box_bottom_right
box_top_left
reward_port
box_bottom_left
agent
baseline_start_timestamp
post_trial_end_timestamp
video_frame
trial_agent_locations
trial_agent_thorax_to_reward_port
trial_agent_thorax_velocity
trial_frame_index
trial_subject_locations
trial_subject_thorax_to_reward_po

# Filtering out spikes

In [77]:
TRIALS_AND_SPECTRAL_DF["baseline_spike_times"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: filter_spike_times(x["spike_times"], start=x["baseline_start_timestamp"], stop=x["tone_start_timestamp"]).astype(int), axis=1)


In [78]:
TRIALS_AND_SPECTRAL_DF["trial_spike_times"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: filter_spike_times(x["spike_times"], start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"]).astype(int), axis=1)


In [79]:
TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"].head()

0    1275889
1    1275889
2    3275911
3    3275911
4    4475927
Name: tone_start_timestamp, dtype: int64

In [80]:
TRIALS_AND_SPECTRAL_DF["baseline_start_timestamp"].head()

0    1075889
1    1075889
2    3075911
3    3075911
4    4275927
Name: baseline_start_timestamp, dtype: int64

In [81]:
TRIALS_AND_SPECTRAL_DF["baseline_spike_times"].iloc[0]

array([[1103733, 1122516, 1123079, ...,      -1,      -1,      -1],
       [1082337, 1084207, 1085898, ...,      -1,      -1,      -1],
       [1158168,      -1,      -1, ...,      -1,      -1,      -1],
       ...,
       [1088690, 1089366, 1093440, ...,      -1,      -1,      -1],
       [1078045, 1080487, 1081051, ...,      -1,      -1,      -1],
       [1082307, 1096129, 1120683, ...,      -1,      -1,      -1]])

In [82]:
TRIALS_AND_SPECTRAL_DF["baseline_neuron_average_fr"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["tone_start_timestamp"], timestamps=x["neuron_average_timestamps"], items=x["neuron_average_fr"].T)[1], axis=1)
TRIALS_AND_SPECTRAL_DF["baseline_neuron_average_timestamp"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["tone_start_timestamp"], timestamps=x["neuron_average_timestamps"], items=x["neuron_average_fr"].T)[0], axis=1)

In [83]:
TRIALS_AND_SPECTRAL_DF["trial_neuron_average_fr"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["neuron_average_timestamps"], items=x["neuron_average_fr"].T)[1].T, axis=1)
TRIALS_AND_SPECTRAL_DF["trial_neuron_average_timestamp"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["neuron_average_timestamps"], items=x["neuron_average_fr"].T)[0], axis=1)

In [84]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=["spike_clusters", "spike_times", "neuron_average_fr", "neuron_average_timestamps",], errors="ignore")

In [85]:
TRIALS_AND_SPECTRAL_DF["trial_neuron_average_fr"].iloc[0].shape

(15, 100)

In [86]:
TRIALS_AND_SPECTRAL_DF["trial_neuron_average_timestamp"].iloc[0].shape

(100,)

- Filtering coherence

In [87]:
coherence_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "coherence" in col and "timestamps" not in col and "calculation" not in col]

In [88]:
coherence_columns

[]

In [89]:
try:
    for col in sorted(coherence_columns):
        updated_item_col = "trial_{}".format(col)
        print(updated_item_col)
        updated_timestamp_col = "trial_coherence_timestamps"
        TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["coherence_timestamps"], items=x[col])[1], axis=1)

    TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["coherence_timestamps"], items=x[col])[0], axis=1)
except:
    pass

In [90]:
try:
    for col in sorted(coherence_columns):
        updated_item_col = "baseline_{}".format(col)
        print(updated_item_col)
        updated_timestamp_col = "baseline_coherence_timestamps"
        TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["tone_start_timestamp"], timestamps=x["coherence_timestamps"], items=x[col])[1], axis=1)

    TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["tone_start_timestamp"], timestamps=x["coherence_timestamps"], items=x[col])[0], axis=1)
except:
    pass

In [91]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=coherence_columns + ["coherence_timestamps"], errors="ignore")

- Filtering Grangers

In [92]:
granger_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "granger" in col and "timestamps" not in col and "calculation" not in col]

In [93]:
granger_columns

[]

In [94]:
try:
    for col in sorted(granger_columns):
        updated_item_col = "trial_{}".format(col)
        print(updated_item_col)
        updated_timestamp_col = "trial_granger_timestamps"
        TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["granger_timestamps"], items=x[col])[1], axis=1)

    TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["granger_timestamps"], items=x[col])[0], axis=1)
except:
    pass

In [95]:
try:
    for col in sorted(granger_columns):
        updated_item_col = "baseline_{}".format(col)
        print(updated_item_col)
        updated_timestamp_col = "baseline_granger_timestamps"
        TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["tone_start_timestamp"], timestamps=x["granger_timestamps"], items=x[col])[1], axis=1)

    TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["tone_start_timestamp"], timestamps=x["granger_timestamps"], items=x[col])[0], axis=1)
except:
    pass

In [96]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=granger_columns + ["granger_timestamps"], errors="ignore")

- Filtering power

In [97]:
TRIALS_AND_SPECTRAL_DF.head()

,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,experiment,video_name,...,baseline_subject_locations,baseline_subject_thorax_to_reward_port,baseline_subject_thorax_velocity,baseline_video_timestamps,baseline_spike_times,trial_spike_times,baseline_neuron_average_fr,baseline_neuron_average_timestamp,trial_neuron_average_fr,trial_neuron_average_timestamp
0,1272,1352,1372.0,1471,rewarded,NaN,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,"[[[53.554223881353295, 34.012170346436235], [5...","[20.08, 20.05, 20.02, 19.98, 19.95, 19.92, 19....","[0.765, 0.765, 0.765, 0.765, 0.765, 0.765, 0.7...","[1076819, 1078205, 1078205, 1079591, 1080976, ...","[[1103733, 1122516, 1123079, 1197152, 1238005,...","[[1278428, 1288446, 1299034, 1307410, 1349996,...","[[0.0, 0.0, 0.0, 6.0, 1.0, 0.0, 0.0, 0.0, 0.0,...","[1076000, 1078000, 1080000, 1082000, 1084000, ...","[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0,...","[1276000, 1278000, 1280000, 1282000, 1284000, ..."
1,1272,1352,1372.0,1471,rewarded,NaN,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,"[[[55.830406542433366, 14.855499448897977], [5...","[23.42, 23.38, 23.16, 23.14, 23.14, 23.05, 23....","[0.732, 0.8413, 0.997, 1.205, 1.408, 1.698, 2....","[1076819, 1078205, 1078205, 1079591, 1080976, ...","[[1083272, 1085299, 1092366, 1096039, 1109069,...","[[1277150, 1287988, 1291074, 1291788, 1296855,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[1076000, 1078000, 1080000, 1082000, 1084000, ...","[[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 2.0, 0.0,...","[1276000, 1278000, 1280000, 1282000, 1284000, ..."
2,3268,3348,3368.0,3469,rewarded,NaN,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,"[[[72.31235192301608, 12.466513975820124], [68...","[1.71, 1.708, 1.707, 1.706, 1.705, 1.703, 1.70...","[0.03372, 0.03372, 0.03372, 0.03372, 0.03372, ...","[3076629, 3076629, 3078015, 3079400, 3080786, ...","[[3080416, 3139037, 3141153, 3143036, 3144932,...","[[3303781, 3359769, 3363493, 3388043, 3401743,...","[[0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 2.0,...","[3076000, 3078000, 3080000, 3082000, 3084000, ...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[3276000, 3278000, 3280000, 3282000, 3284000, ..."
3,3268,3348,3368.0,3469,rewarded,NaN,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,"[[[65.04972718833214, 33.63010427744915], [64....","[8.63, 8.58, 9.04, 8.695, 7.91, 8.19, 7.88, 7....","[7.152, 6.49, 5.996, 5.605, 5.305, 5.04, 4.793...","[3076629, 3076629, 3078015, 3079400, 3080786, ...","[[3080579, 3083556, 3091249, 3092518, 3103212,...","[[3280834, 3282844, 3284986, 3286979, 3289006,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[3076000, 3078000, 3080000, 3082000, 3084000, ...","[[0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 2.0, 0.0, 1.0,...","[3276000, 3278000, 3280000, 3282000, 3284000, ..."
4,4466,4546,4566.0,4667,rewarded,NaN,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,"[[[60.31504673710292, 35.65648479273409], [57....","[1.177, 1.178, 1.178, 1.179, 1.18, 1.18, 1.181...","[0.03372, 0.03372, 0.03372, 0.03372, 0.03372, ...","[4276792, 4278178, 4278178, 4279563, 4280949, ...","[[4373339, -1, -1, -1, -1, -1, -1, -1, -1, -1,...","[[4481591, 4495795, 4631112, 4632893, -1, -1, ...","[[0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 1.0, 1.0, 0.0,...","[4276000, 4278000, 4280000, 4282000, 4284000, ...","[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[4476000, 4478000, 4480000, 4482000, 4484000, ..."


In [98]:
power_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "power" in col and "timestamps" not in col and "calculation" not in col]

In [99]:
power_columns

[]

In [100]:
try:
    for col in sorted(power_columns):
        updated_item_col = "trial_{}".format(col)
        print(updated_item_col)
        updated_timestamp_col = "trial_power_timestamps"
        TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["power_timestamps"], items=x[col])[1], axis=1)

    TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["power_timestamps"], items=x[col])[0], axis=1)
except:
    pass

In [101]:
try:
    for col in sorted(power_columns):
        updated_item_col = "baseline_{}".format(col)
        print(updated_item_col)
        updated_timestamp_col = "baseline_power_timestamps"
        TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["tone_start_timestamp"], timestamps=x["power_timestamps"], items=x[col])[1], axis=1)

    TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["tone_start_timestamp"], timestamps=x["power_timestamps"], items=x[col])[0], axis=1)
except:
    pass

In [102]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=power_columns + ["power_timestamps"], errors="ignore")

# Filtering out phase

In [103]:
lfp_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "trace" in col and "timestamps" not in col and "calculation" not in col]

In [104]:
band_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "band" in col and "timestamps" not in col and "calculation" not in col]

In [105]:
phase_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "phase" in col and "timestamps" not in col and "calculation" not in col]

In [106]:
phase_columns = phase_columns + band_columns +lfp_columns

In [107]:
phase_columns

['vHPC_lfp_trace',
 'BLA_lfp_trace',
 'LH_lfp_trace',
 'MD_lfp_trace',
 'mPFC_lfp_trace']

In [108]:
try:
    for col in sorted(phase_columns):
        updated_item_col = "trial_{}".format(col)
        print(updated_item_col)
        updated_timestamp_col = "trial_lfp_timestamps"
        TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["lfp_timestamps"], items=x[col])[1], axis=1)

    TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["lfp_timestamps"], items=x[col])[0], axis=1)
except:
    pass

trial_BLA_lfp_trace
trial_LH_lfp_trace
trial_MD_lfp_trace
trial_mPFC_lfp_trace
trial_vHPC_lfp_trace


In [109]:
try:
    for col in sorted(phase_columns):
        updated_item_col = "baseline_{}".format(col)
        print(updated_item_col)
        updated_timestamp_col = "baseline_lfp_timestamps"
        TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["tone_start_timestamp"], timestamps=x["lfp_timestamps"], items=x[col])[1], axis=1)

    TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["tone_start_timestamp"], timestamps=x["lfp_timestamps"], items=x[col])[0], axis=1)
except:
    pass

baseline_BLA_lfp_trace
baseline_LH_lfp_trace
baseline_MD_lfp_trace
baseline_mPFC_lfp_trace
baseline_vHPC_lfp_trace


In [110]:
TRIALS_AND_SPECTRAL_DF["trial_lfp_timestamps"]

0      [1275900, 1275920, 1275940, 1275960, 1275980, ...
1      [1275900, 1275920, 1275940, 1275960, 1275980, ...
2      [3275920, 3275940, 3275960, 3275980, 3276000, ...
3      [3275920, 3275940, 3275960, 3275980, 3276000, ...
4      [4475940, 4475960, 4475980, 4476000, 4476020, ...
                             ...                        
454    [55564880, 55564900, 55564920, 55564940, 55564...
455    [56564880, 56564900, 56564920, 56564940, 56564...
456    [57964900, 57964920, 57964940, 57964960, 57964...
457    [60164920, 60164940, 60164960, 60164980, 60165...
458    [61264940, 61264960, 61264980, 61265000, 61265...
Name: trial_lfp_timestamps, Length: 459, dtype: object

In [111]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=phase_columns + ["lfp_timestamps"], errors="ignore")

In [112]:
for col in TRIALS_AND_SPECTRAL_DF:
    print(col)

tone_start_frame
reward_start
reward_dispensed
tone_stop_frame
condition
competition_closeness
get_reward_frame
out_reward_frame
experiment
video_name
tone_start_timestamp
tone_stop_timestamp
cohort
session_dir
tone_frames
box_1_port_entry_frames
box_2_port_entry_frames
notes
session_path
recording
current_subject
subject
all_subjects
first_timestamp
last_timestamp
tone_timestamps
box_1_port_entry_timestamps
box_2_port_entry_timestamps
recording_name
start_frame
stop_frame
tracked_subject
in_video_subjects
box_number
sleap_name
body_parts
box_top_right
box_bottom_right
box_top_left
reward_port
box_bottom_left
agent
baseline_start_timestamp
post_trial_end_timestamp
video_frame
trial_agent_locations
trial_agent_thorax_to_reward_port
trial_agent_thorax_velocity
trial_frame_index
trial_subject_locations
trial_subject_thorax_to_reward_port
trial_subject_thorax_velocity
trial_video_timestamps
baseline_agent_locations
baseline_agent_thorax_to_reward_port
baseline_agent_thorax_velocity
baselin

In [113]:
# TRIALS_AND_SPECTRAL_DF[TRIALS_AND_SPECTRAL_DF["condition"] != "rewarded"]["trial_kmeans_cluster"].iloc[0].shape

In [114]:
TRIALS_AND_SPECTRAL_DF.shape

(459, 79)

In [115]:
TRIALS_AND_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

# Filtering for rows that are in the video

In [116]:
TRIALS_AND_SPECTRAL_DF["in_video"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: x["start_frame"] <= x["trial_frame_index"][0] <= x["stop_frame"], axis=1)

In [117]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF[TRIALS_AND_SPECTRAL_DF["in_video"]].reset_index()

In [118]:
TRIALS_AND_SPECTRAL_DF

,index,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,experiment,...,trial_mPFC_lfp_trace,trial_vHPC_lfp_trace,trial_lfp_timestamps,baseline_BLA_lfp_trace,baseline_LH_lfp_trace,baseline_MD_lfp_trace,baseline_mPFC_lfp_trace,baseline_vHPC_lfp_trace,baseline_lfp_timestamps,in_video
0,42,34240,34320,34340.0,34440,3.1,Subj 1 Only,NaN,NaN,alone_comp,...,"[601, 404, -26, -167, 375, 672, 269, 92, 346, ...","[1075, 747, 267, 86, 601, 910, 618, 610, 948, ...","[34276300, 34276320, 34276340, 34276360, 34276...","[351, 112, -131, -294, -402, -370, -53, 215, 2...","[283, 41, -194, -246, -216, -171, -4, 192, 234...","[-22, -191, -363, -383, -289, -233, -161, -12,...","[54, -7, -78, -114, -138, -173, -116, 14, 0, -...","[-664, -728, -689, -550, -406, -206, 101, 320,...","[34076300, 34076320, 34076340, 34076360, 34076...",True
1,43,34240,34320,34340.0,34440,3.1,Subj 1 Only,NaN,NaN,alone_comp,...,"[1009, 845, 573, 278, 116, -59, -640, -1498, -...","[1334, 1214, 1026, 724, 446, 202, -388, -1229,...","[34276300, 34276320, 34276340, 34276360, 34276...","[-327, -222, -95, -40, -10, 106, 228, 192, 47,...","[-587, -450, -274, -165, -131, -83, -51, -139,...","[-123, -44, -10, -58, -54, 66, 192, 197, 112, ...","[186, 251, 325, 302, 258, 295, 298, 222, 230, ...","[-349, -354, -349, -327, -281, -214, -164, -16...","[34076300, 34076320, 34076340, 34076360, 34076...",True
2,44,35238,35318,35338.0,35437,3.1,Subj 1 then Subj 2,NaN,NaN,alone_comp,...,"[500, 532, 470, 436, 418, 408, 390, 291, 188, ...","[-1269, -1286, -1269, -1248, -1267, -1326, -13...","[35276320, 35276340, 35276360, 35276380, 35276...","[-568, -481, -424, -573, -790, -764, -497, -24...","[-578, -368, -126, -160, -371, -386, -179, -80...","[-260, -192, -73, -74, -297, -401, -197, -84, ...","[-1222, -935, -749, -822, -1002, -949, -668, -...","[-3143, -2828, -2568, -2527, -2642, -2675, -25...","[35076320, 35076340, 35076360, 35076380, 35076...",True
3,45,35238,35318,35338.0,35437,3.1,Subj 1 then Subj 2,NaN,NaN,alone_comp,...,"[353, 480, 674, 856, 910, 950, 1083, 1251, 127...","[-681, -526, -258, 42, 230, 315, 427, 566, 588...","[35276320, 35276340, 35276360, 35276380, 35276...","[-715, -527, -698, -933, -1050, -1050, -1055, ...","[-865, -668, -859, -1021, -1029, -1021, -1050,...","[-593, -349, -471, -727, -909, -986, -1044, -1...","[-661, -502, -705, -998, -1192, -1159, -1083, ...","[-1643, -1233, -1132, -1203, -1290, -1368, -14...","[35076320, 35076340, 35076360, 35076380, 35076...",True
4,46,36337,36417,36437.0,36536,3.3,Subj 2 Only,NaN,NaN,alone_comp,...,"[539, 510, 567, 462, 112, -106, 53, 245, 288, ...","[-1868, -1989, -1865, -1712, -1624, -1487, -12...","[36376340, 36376360, 36376380, 36376400, 36376...","[-687, -630, -622, -525, -257, -149, -393, -68...","[-340, -256, -445, -630, -490, -308, -440, -72...","[-310, -228, -491, -779, -698, -583, -794, -10...","[-505, -451, -691, -795, -607, -539, -837, -12...","[655, 500, 190, -26, 49, 85, -205, -601, -749,...","[36176340, 36176360, 36176380, 36176400, 36176...",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,454,55484,55564,55584.0,55685,4.4,Subj 2 then Subj 1,NaN,NaN,alone_comp,...,"[467, 372, 334, 349, 295, 182, 92, 38, -52, -1...","[1066, 932, 943, 886, 607, 236, -87, -378, -75...","[55564880, 55564900, 55564920, 55564940, 55564...","[22, -72, -33, 105, 120, -9, -108, -63, 23, 10...","[-175, -184, -18, 58, -103, -153, -2, 67, 43, ...","[176, 198, 377, 469, 367, 332, 327, 221, 169, ...","[-349, -307, -181, -217, -398, -427, -371, -36...","[-892, -566, -151, 136, 95, -24, 31, 210, 395,...","[55364880, 55364900, 55364920, 55364940, 55364...",True
224,455,56483,56563,56583.0,56683,4.4,Subj 2 then Subj 1,NaN,NaN,alone_comp,...,"[193, 267, 317, 217, 131, 323, 564, 502, 354, ...","[-1039, -1125, -1275, -1576, -1844, -1846, -17...","[56564880, 56564900, 56564920, 56564940, 56564...","[37, 133, 191, 201, 228, 282, 2

In [119]:
TRIALS_AND_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

In [120]:
raise ValueError()

ValueError: 